<a href="https://colab.research.google.com/github/Yujini68/ESAA/blob/main/%EB%AF%B8%EB%8B%88%ED%94%84%EB%A1%9C%EC%A0%9D%ED%8A%B82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#라이브러리 임포트
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

#경고 메시지 제외
import warnings
warnings.filterwarnings(action='ignore')

#데이터 불러오기
train = pd.read_csv('/content/train.csv')
test = pd.read_csv('/content/test.csv')
sample = pd.read_csv('/content/sample_submission.csv')

# 원-핫 인코딩

In [2]:
train=train.drop('index', axis=1)
test=test.drop('index', axis=1)

In [3]:
train=pd.get_dummies(data=train, columns=['type'], prefix=['type'], dtype=int)

In [4]:
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_red,type_white
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0,1
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,1,0
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0,1
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0,1
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0,1


In [5]:
test=pd.get_dummies(data=test, columns=['type'], prefix=['type'], dtype=int)

In [6]:
test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type_red,type_white
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,0,1
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,1,0
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,0,1
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,0,1
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,0,1


# LightGBM

In [7]:
train_y = train['quality']
train_x=train.drop('quality', axis=1)

In [8]:
# 데이터 split
from sklearn.model_selection import GridSearchCV, train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, shuffle = True,
                                                    test_size = 0.2, random_state=156)

# 모델 생성
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier(objective='multiclass', metric='multi_logloss', verbosity=-1)

# 모델 학습
params = {'n_estimators': [80, 90, 100, 110, 120],
          'learning_rate': [0.08, 0.09, 0.1, 0.11, 0.12],
          'reg_alpha': [0.88, 0.9, 0.92]}

gridcv = GridSearchCV(lgbm_clf, param_grid=params, scoring='accuracy', cv=5, verbose=1, error_score='raise')
gridcv.fit(X_train, y_train, eval_metric='logloss', eval_set=[(X_train, y_train), (X_test, y_test)])
print("GridSearchCV 최고 정확도: {0:.4f}".format(gridcv.best_score_))
print("GridSearchCV 최적 파라미터:", gridcv.best_params_)

Fitting 5 folds for each of 75 candidates, totalling 375 fits
GridSearchCV 최고 정확도: 0.6382
GridSearchCV 최적 파라미터: {'learning_rate': 0.1, 'n_estimators': 110, 'reg_alpha': 0.88}


In [10]:
# 최적 파라미터 대입
from sklearn.metrics import accuracy_score
lgbm_wrapper = LGBMClassifier(learning_rate=0.1, n_estimators=110, reg_alpha=0.88, objective='multiclass', metric='multi_logloss', verbosity=-1)

lgbm_wrapper.fit(X_train, y_train, eval_metric='logloss',
                eval_set=[(X_train, y_train), (X_test, y_test)])
predictions = lgbm_wrapper.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f'예측 정확도: {accuracy:.4f}')

예측 정확도: 0.6755


# test 데이터 예측

In [11]:
# 제출 파일 생성
sub = lgbm_wrapper.predict(test)
submission = pd.read_csv('/content/sample_submission.csv')
submission['quality']= sub
submission.head()

,index,quality
0,0,5
1,1,5
2,2,6
3,3,5
4,4,6


In [13]:
# csv 파일로 저장
submission.to_csv('submission_LGBM_grid.csv', index=False)

# SMOTE 데이터 모델 생성

In [14]:
train_y.value_counts()

quality
6    2416
5    1788
7     924
4     186
8     152
3      26
9       5
Name: count, dtype: int64

레이블 불균형 -> SMOTE 적용

In [15]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(k_neighbors = 4, random_state = 0)
X_train_over, y_train_over = smote.fit_resample(train_x, train_y)
print('SMOTE 적용 전 학습용 피처/레이블 데이터 세트: ', train_x.shape, train_y.shape)
print('SMOTE 적용 후 학습용 피처/레이블 데이터 세트: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 레이블 값 분포: \n', pd.Series(y_train_over).value_counts())

SMOTE 적용 전 학습용 피처/레이블 데이터 세트:  (5497, 13) (5497,)
SMOTE 적용 후 학습용 피처/레이블 데이터 세트:  (16912, 13) (16912,)
SMOTE 적용 후 레이블 값 분포: 
 quality
5    2416
6    2416
7    2416
8    2416
4    2416
3    2416
9    2416
Name: count, dtype: int64


In [17]:
# 모델 생성
from lightgbm import LGBMClassifier
lgbm_smote = LGBMClassifier(objective='multiclass', metric='multi_logloss', verbosity=-1)

# 모델 학습
params = {'n_estimators': [80, 90, 100, 110, 120],
          'learning_rate': [0.08, 0.09, 0.1, 0.11, 0.12],
          'reg_alpha': [0.88, 0.9, 0.92]}

gridcv_smote = GridSearchCV(lgbm_smote, param_grid=params, scoring='accuracy', cv=5, verbose=1, error_score='raise')
gridcv_smote.fit(X_train_over, y_train_over, eval_metric='logloss')
print("GridSearchCV 최고 정확도: {0:.4f}".format(gridcv_smote.best_score_))
print("GridSearchCV 최적 파라미터:", gridcv_smote.best_params_)

Fitting 5 folds for each of 75 candidates, totalling 375 fits
GridSearchCV 최고 정확도: 0.8790
GridSearchCV 최적 파라미터: {'learning_rate': 0.12, 'n_estimators': 120, 'reg_alpha': 0.9}


In [21]:
# 최적 파라미터 대입
from sklearn.metrics import accuracy_score
lgbm_wrapper_smote = LGBMClassifier(learning_rate=0.12, n_estimators=120, reg_alpha=0.9, objective='multiclass', metric='multi_logloss', verbosity=-1)

lgbm_wrapper_smote.fit(X_train_over, y_train_over, eval_metric='logloss')
predictions = lgbm_wrapper_smote.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print(f'예측 정확도: {accuracy:.4f}')

예측 정확도: 0.9545


# test 데이터 예측

In [22]:
# 제출 파일 생성
sub = lgbm_wrapper_smote.predict(test)
submission = pd.read_csv('/content/sample_submission.csv')
submission['quality']= sub
submission.head()

,index,quality
0,0,5
1,1,6
2,2,5
3,3,5
4,4,7


In [24]:
# csv 파일로 저장
submission.to_csv('submission_LGBM_smote.csv', index=False)